In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-demand/train.csv
/kaggle/input/predict-demand/test.csv


In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df=pd.read_csv('../input/predict-demand/train.csv')
test_df=pd.read_csv('../input/predict-demand/test.csv')

In [4]:
train_df

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,glass,500ml,0.96,13280.0
1,1.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,plastic,1.5lt,2.86,6727.0
2,2.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,kinder-cola,can,330ml,0.87,9848.0
3,3.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,glass,500ml,1.00,20050.0
4,4.0,31/01/12,Athens,37.97945,23.71622,672130.0,shop_1,adult-cola,can,330ml,0.39,25696.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
test_df

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,6480,31/01/18,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,1.5lt,3.10,7056
1,6481,31/01/18,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,can,330ml,0.85,12490
2,6482,31/01/18,Athens,37.97945,23.71622,664046,shop_1,adult-cola,glass,500ml,0.83,26640
3,6483,31/01/18,Athens,37.97945,23.71622,664046,shop_1,orange-power,glass,500ml,0.54,41892
4,6484,31/01/18,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,1.5lt,0.83,22923
...,...,...,...,...,...,...,...,...,...,...,...,...
1075,7555,31/12/18,Athens,37.97945,23.71622,664046,shop_1,kinder-cola,plastic,1.5lt,2.52,13760
1076,7556,31/12/18,Athens,37.97945,23.71622,664046,shop_1,orange-power,plastic,1.5lt,2.18,16309
1077,7557,31/12/18,Patra,38.24444,21.73444,168034,shop_6,kinder-cola,can,330ml,0.85,24378
1078,7558,31/12/18,Thessaloniki,40.64361,22.93086,354290,shop_4,adult-cola,plastic,1.5lt,2.17,20691


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7560 entries, 0 to 7559
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         6480 non-null   float64
 1   date       6480 non-null   object 
 2   city       6480 non-null   object 
 3   lat        6429 non-null   float64
 4   long       6434 non-null   float64
 5   pop        6480 non-null   float64
 6   shop       6480 non-null   object 
 7   brand      6480 non-null   object 
 8   container  6464 non-null   object 
 9   capacity   6465 non-null   object 
 10  price      6480 non-null   float64
 11  quantity   6480 non-null   float64
dtypes: float64(6), object(6)
memory usage: 708.9+ KB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1080 non-null   int64  
 1   date       1080 non-null   object 
 2   city       1080 non-null   object 
 3   lat        1072 non-null   float64
 4   long       1067 non-null   float64
 5   pop        1080 non-null   int64  
 6   shop       1080 non-null   object 
 7   brand      1080 non-null   object 
 8   container  1077 non-null   object 
 9   capacity   1076 non-null   object 
 10  price      1080 non-null   float64
 11  quantity   1080 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 101.4+ KB


Preprocessing

In [8]:
#splitting columns into dummies
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df
#splitting date
def encode_dates(df, column):
    df = df.copy()
    df[column] = pd.to_datetime(df[column])
    df[column +'_year'] = df[column].apply(lambda x: x.year)
    df[column +'_month'] = df[column].apply(lambda x: x.month)
    df[column +'_day'] = df[column].apply(lambda x: x.day)
    df = df.drop(column, axis=1)
    return df

In [9]:
def preprocess_inputs(df):
    df = df.copy()
    
    #drop id column
    df=df.drop('id', axis=1)
    
    #remove missing rows
    missing_rows=df.loc[df.isna().all(axis=1), :].index
    df =df.drop(missing_rows, axis=0).reset_index(drop=True) #null values in lat,long,capacity, container)
    
    #fill numeric missing values with mean
    for column in ['lat', 'long']:
        df[column]=df[column].fillna(df[column].mean())
    
    #Fill ordinal missing values with mode (frequent values used to replace null values i.e 1.5lt)
    df['capacity']=df['capacity'].fillna(df['capacity'].mode()[0])
    
    #Ordinal encode capacity column (330=0,500=1,1.5lt=2)
    capacity_ordering = ['330ml', '500ml', '1.5lt']
    df['capacity']= df['capacity'].apply(lambda x: capacity_ordering.index(x))
    
    #one-hot encode nominal features (changing categorical into numerical)
    for column in ['city', 'shop', 'brand', 'container']:
        df= onehot_encode(df, column=column)
    
    #Extract date features (splitting day,month and year in date)
    df = encode_dates(df, column='date')
    
    #Split df into X and y
    y=df['quantity']
    X=df.drop('quantity', axis=1)
                                                                      

    return X, y

In [10]:
X_train,y_train= preprocess_inputs(train_df)
X_test,y_test= preprocess_inputs(test_df)

scaler = StandardScaler()
scaler.fit(X_train)

X_train=pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test=pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)

In [11]:
X_train

,lat,long,pop,capacity,price,city_Athens,city_Irakleion,city_Larisa,city_Patra,city_Thessaloniki,...,brand_gazoza,brand_kinder-cola,brand_lemon-boost,brand_orange-power,container_can,container_glass,container_plastic,date_year,date_month,date_day
0,-0.195837,0.411791,1.361571,-0.003021,-0.281130,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,2.0,-0.5,-0.5,-0.705389,1.417165,-0.706370,-1.46385,-1.593255,0.696733
1,-0.195837,0.411791,1.361571,1.220597,2.051577,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,2.0,-0.5,-0.5,-0.705389,-0.705634,1.415688,-1.46385,-1.593255,0.696733
2,-0.195837,0.411791,1.361571,-1.226639,-0.391627,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,2.0,-0.5,-0.5,1.417658,-0.705634,-0.706370,-1.46385,-1.593255,0.696733
3,-0.195837,0.411791,1.361571,-0.003021,-0.232020,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,-0.5,-0.705389,1.417165,-0.706370,-1.46385,-1.593255,0.696733
4,-0.195837,0.411791,1.361571,-1.226639,-0.980942,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,-0.5,1.417658,-0.705634,-0.706370,-1.46385,-1.593255,0.696733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,-0.206183,0.384885,1.334687,1.220597,-0.207465,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,2.0,-0.705389,-0.705634,1.415688,1.46385,1.593255,0.696733
6476,0.812867,-0.787282,-0.905625,-1.226639,-0.882723,-0.707107,-0.447214,2.236068,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,2.0,1.417658,-0.705634,-0.706370,1.46385,1.593255,0.696733
6477,-0.034567,-1.418088,-0.801683,-0.003021,-0.207465,-0.707107,-0.447214,-0.447214,2.236068,-0.447214,...,-0.5,-0.5,-0.5,-0.5,-0.705389,1.417165,-0.706370,1.46385,1.593255,0.696733
6478,1.425548,-0.313372,-0.009194,1.220597,0.185412,-0.707107,-0.447214,-0.447214,-0.447214,2.236068,...,2.0,-0.5,-0.5,-0.5,-0.705389,-0.705634,1.415688,1.46385,1.593255,0.696733


In [12]:
X_train['capacity'].unique()

array([-0.00302128,  1.22059673, -1.22663928])

In [13]:
['500ml', '1.5lt', '330ml'].index('1.5lt')

1

In [14]:
X_train['capacity'].mode()

0    1.220597
dtype: float64

In [15]:
X_train['capacity'].mode()[0]

1.2205967265736783

In [16]:
X_train.isna().sum()

lat                   0
long                  0
pop                   0
capacity              0
price                 0
city_Athens           0
city_Irakleion        0
city_Larisa           0
city_Patra            0
city_Thessaloniki     0
shop_shop_1           0
shop_shop_2           0
shop_shop_3           0
shop_shop_4           0
shop_shop_5           0
shop_shop_6           0
brand_adult-cola      0
brand_gazoza          0
brand_kinder-cola     0
brand_lemon-boost     0
brand_orange-power    0
container_can         0
container_glass       0
container_plastic     0
date_year             0
date_month            0
date_day              0
dtype: int64

In [17]:
X_test

,lat,long,pop,capacity,price,city_Athens,city_Irakleion,city_Larisa,city_Patra,city_Thessaloniki,...,brand_gazoza,brand_kinder-cola,brand_lemon-boost,brand_orange-power,container_can,container_glass,container_plastic,date_year,date_month,date_day
0,-0.195837,0.411791,1.326848,1.220597,2.346235,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,2.0,-0.5,-0.5,-0.705389,-0.705634,1.415688,2.04939,-1.593255,0.696733
1,-0.195837,0.411791,1.326848,-1.226639,-0.416181,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,2.0,-0.5,-0.5,1.417658,-0.705634,-0.706370,2.04939,-1.593255,0.696733
2,-0.195837,0.411791,1.326848,-0.003021,-0.440736,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,-0.5,-0.705389,1.417165,-0.706370,2.04939,-1.593255,0.696733
3,-0.195837,0.411791,1.326848,-0.003021,-0.796781,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,2.0,-0.705389,1.417165,-0.706370,2.04939,-1.593255,0.696733
4,-0.195837,0.411791,1.326848,1.220597,-0.440736,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,2.0,-0.705389,-0.705634,1.415688,2.04939,-1.593255,0.696733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,-0.195837,0.411791,1.326848,1.220597,1.634146,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,2.0,-0.5,-0.5,-0.705389,-0.705634,1.415688,2.04939,1.593255,0.696733
1076,-0.195837,0.411791,1.326848,1.220597,1.216714,1.414214,-0.447214,-0.447214,-0.447214,-0.447214,...,-0.5,-0.5,-0.5,2.0,-0.705389,-0.705634,1.415688,2.04939,1.593255,0.696733
1077,-0.034567,-1.418088,-0.803689,-1.226639,-0.416181,-0.707107,-0.447214,-0.447214,2.236068,-0.447214,...,-0.5,2.0,-0.5,-0.5,1.417658,-0.705634,-0.706370,2.04939,1.593255,0.696733
1078,1.425548,-0.313372,-0.003657,1.220597,1.204436,-0.707107,-0.447214,-0.447214,-0.447214,2.236068,...,-0.5,-0.5,-0.5,-0.5,-0.705389,-0.705634,1.415688,2.04939,1.593255,0.696733


In [18]:
X_test.isna().sum()

lat                   0
long                  0
pop                   0
capacity              0
price                 0
city_Athens           0
city_Irakleion        0
city_Larisa           0
city_Patra            0
city_Thessaloniki     0
shop_shop_1           0
shop_shop_2           0
shop_shop_3           0
shop_shop_4           0
shop_shop_5           0
shop_shop_6           0
brand_adult-cola      0
brand_gazoza          0
brand_kinder-cola     0
brand_lemon-boost     0
brand_orange-power    0
container_can         0
container_glass       0
container_plastic     0
date_year             0
date_month            0
date_day              0
dtype: int64

In [19]:
y_test

0        7056
1       12490
2       26640
3       41892
4       22923
        ...  
1075    13760
1076    16309
1077    24378
1078    20691
1079    24615
Name: quantity, Length: 1080, dtype: int64

In [20]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [21]:
model1= KNeighborsRegressor()
model1.fit(X_train, y_train)
model1.score(X_test, y_test)*100 

68.2752142144272

In [22]:
model2= LinearRegression()
model2.fit(X_train, y_train)
model2.score(X_test, y_test)*100 

57.16539639738516

In [23]:
model3= Ridge()
model3.fit(X_train, y_train)
model3.score(X_test, y_test)*100 

57.24278386882964

In [24]:
model4= LinearSVR()
model4.fit(X_train, y_train)
model4.score(X_test, y_test)*100

-198.52989017339837

In [25]:
model5= DecisionTreeRegressor()
model5.fit(X_train, y_train)
model5.score(X_test, y_test)*100

81.866776402525

In [28]:
model6=RandomForestRegressor()
model6.fit(X_train, y_train)
model6.score(X_test, y_test)*100

92.33418032421146

In [29]:
model7=GradientBoostingRegressor()
model7.fit(X_train, y_train)
model7.score(X_test, y_test)*100

88.48826045881894